In [2]:
import pandas as pd
import re
import os
from string import punctuation
punct = punctuation+'«»—…“”*№–'
import itertools
from razdel import tokenize
import numpy as np
import codecs


regex = re.compile("[А-ЯЁЙ]{3,}") 

## 1. Читаем датасеты и объединяем в 1 датафрейм

In [5]:
filenames = [i for i in os.listdir(".") if i.startswith("capitalization-")]
print(filenames)

['capitalization-lentach-wall.csv', 'capitalization-yajemati-wall.csv', 'capitalization-sssr-wall.csv', 'capitalization-vdud-wall-new.csv', 'capitalization-sci-wall.csv']


In [157]:
df = pd.read_csv("capitalization-lentach-wall.csv")

In [8]:
df.columns.tolist()

['from_id',
 'parent_id',
 'comment_text',
 'comment_id',
 'parent_text',
 'first_name',
 'last_name',
 'sex',
 'bdate',
 'city',
 'group_info',
 'source']

In [6]:
all_dfs = pd.DataFrame(columns = ['from_id',
 'parent_id',
 'comment_text',
 'comment_id',
 'parent_text',
 'first_name',
 'last_name',
 'sex',
 'bdate',
 'city',
 'group_info',
 'source'])

for i in filenames:
    df = pd.read_csv(i)
    all_dfs = all_dfs.append(df, ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
all_dfs.groupby(by="source").from_id.count()

source
lentach     8741
sci         9030
sssr        9591
vdud        2772
yajemati    6613
Name: from_id, dtype: int64

In [8]:
all_dfs.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach
3,18.6.1998,Saint Petersburg,11736884,"[id164270524|Костик], я думаю много весёлого б...",Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach
4,26.7,Novosibirsk,11737428,"[id6343530|Юлия], в какой из бывших республик ...",Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach


In [9]:
all_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36747 entries, 0 to 36746
Data columns (total 12 columns):
bdate           28911 non-null object
city            36737 non-null object
comment_id      36747 non-null object
comment_text    36747 non-null object
first_name      36747 non-null object
from_id         36747 non-null object
group_info      28221 non-null object
last_name       36746 non-null object
parent_id       36747 non-null object
parent_text     24741 non-null object
sex             36747 non-null object
source          36747 non-null object
dtypes: object(12)
memory usage: 3.4+ MB


## 2. Немного очищаем тексты комментариев (избавляем от ссылок на юзеров, хэштегов)

In [10]:
# удалить записи в квадратных скобках
for i in all_dfs.index:
    if re.search("\[id.*|.*\],", all_dfs.loc[i, "comment_text"]):
        all_dfs.loc[i, "comment_text"] = re.sub(r"(\[id.*?|.*?\]), ", r'', all_dfs.loc[i, "comment_text"])

In [11]:
# удалить записи в квадратных скобках
for i in all_dfs.index:
    if re.search("\[club.*|.*\],", all_dfs.loc[i, "comment_text"]):
        all_dfs.loc[i, "comment_text"] = re.sub(r"(\[club.*?|.*?\]), ", r'', all_dfs.loc[i, "comment_text"])

In [12]:
# удалить хэштеги
for i in all_dfs.index:
    if re.search("(#.*?) ", all_dfs.loc[i, "comment_text"]):
        all_dfs.loc[i, "comment_text"] = re.sub("(#.*?) ", '', all_dfs.loc[i, "comment_text"])

In [165]:
all_dfs.tail()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source
36742,10.8.1996,Saint Petersburg,875366,А между тем в РФ ФССП простила населению 2 три...,Konstantin,49836980,Чёрный юмор -||- Самые годные мемы Вконтакте. ...,Belov,875268,NaN,2,sci
36743,NaN,Ulyanovsk,875389,"«Приставы не смогли взыскать в прошлом году 2,...",Dima,64838301,NaN,Dudukin,875268,NaN,2,sci
36744,21.10.1985,Vologda,876004,А у нас переписали определение БЕДНОСТЬ.,Nikolay,10722108,"Comedy Radio -||- Решаем, мутим, разруливаем! ...",Balandin,875268,NaN,2,sci
36745,13.1.1994,Saint Petersburg,877262,"Есть ещё одно НО, это Россия выделит $100 млн ...",Petr,180573259,Manchester United | Манчестер Юнайтед -||- -|...,Potekhin,875268,NaN,2,sci
36746,NaN,Kandalaksha,877188,в США?,Vladimir,6614285,КОНТ. Интересное в мире и интернете -||- Платф...,Zharavin,[875600],Живём в ублюдочной стране.,2,sci


## 3. Добавляем в отдельную колонку число токенов в комменте

In [13]:
# токенизировать и добавить в колонку число слов в комменте (токен - слово больше 2 букв)
comment_texts = all_dfs.comment_text.values.tolist()

In [14]:
comment_texts[0]

'Но это был еще не конец. Я только 31 декабря 1999 года поняла что Гэбня дорвалась до власти и начнется эпоха карлика из КГБ, сворачивание демократии и автократический режим...\nЕще были выборы так называемые....в 2000 году.\nНо и Немцова и Явлинского купили.\nЯвлинский со своей фракцией в Гос.думе мог заблокировать кандидатуру Путина в то время Яблоко имело достаточно голосов для этого\n..и об этом предательстве "демократы" из яблока теперь не вспоминают.\nА Немцов прямо в эфире агитировал за Путина...'

In [16]:
def len_tokens(texts:list):
    tokens_lists = [list(tokenize(i)) for i in texts]
    tokens = [[_.text for _ in tokens if len(_.text) > 2 and _.text not in ["\n", "\t", "  ", "   "]] for tokens in tokens_lists]
    n_tokens = [len(sent) for sent in tokens]
    return tokens, n_tokens

In [17]:
tokens, n_tokens = len_tokens(comment_texts)

In [32]:
max(n_tokens)

1851

In [25]:
n_tokens.index(700)

35641

In [31]:
n_tokens.count(500)

0

In [33]:
all_dfs["n_tokens"] = n_tokens

In [34]:
all_dfs["tokens"] = [" ".join(i) for i in tokens]

In [35]:
all_dfs.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source,n_tokens,tokens
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach,61,это был еще конец только декабря 1999 года пон...
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,6,Лента сама так решила СЛАВУ ИМПЕРАТОРА
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,22,Первый срок часть второго даже одобрял его дей...
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach,26,думаю много весёлого было Например когда 94-ом...
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach,7,какой бывших республик СССР была проведена люс...


In [36]:
df.tail(5)

,from_id,comment_id,comment_text,parent_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
9025,49836980,875366,А между тем в РФ ФССП простила населению 2 три...,875268,NaN,Konstantin,Belov,2,10.8.1996,Saint Petersburg,Чёрный юмор -||- Самые годные мемы Вконтакте. ...,sci
9026,64838301,875389,"«Приставы не смогли взыскать в прошлом году 2,...",875268,NaN,Dima,Dudukin,2,NaN,Ulyanovsk,NaN,sci
9027,10722108,876004,А у нас переписали определение БЕДНОСТЬ.,875268,NaN,Nikolay,Balandin,2,21.10.1985,Vologda,"Comedy Radio -||- Решаем, мутим, разруливаем! ...",sci
9028,180573259,877262,"Есть ещё одно НО, это Россия выделит $100 млн ...",875268,NaN,Petr,Potekhin,2,13.1.1994,Saint Petersburg,Manchester United | Манчестер Юнайтед -||- -|...,sci
9029,6614285,877188,"[id289068065|Ink], в США?",[875600],Живём в ублюдочной стране.,Vladimir,Zharavin,2,NaN,Kandalaksha,КОНТ. Интересное в мире и интернете -||- Платф...,sci


## 4. Также добавляем колонки с числом слов, написанных капсом, и самими словами

In [37]:
# добавить в колонку число слов, написанных капсом; сами эти слова
def len_caps(texts:list):
    tokens_lists = [list(tokenize(i)) for i in texts]
    caps = [[_.text for _ in tokens if (len(_.text) > 2 and re.search(regex, _.text))] for tokens in tokens_lists]
    n_caps = [len(sent) for sent in caps]
    return caps, n_caps

In [38]:
caps, n_caps = len_caps(comment_texts)

In [39]:
all_dfs["n_caps"] = n_caps
all_dfs["caps"] = [" ".join(i) for i in caps]

In [40]:
all_dfs.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source,n_tokens,tokens,n_caps,caps
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach,61,это был еще конец только декабря 1999 года пон...,1,КГБ
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,6,Лента сама так решила СЛАВУ ИМПЕРАТОРА,2,СЛАВУ ИМПЕРАТОРА
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,22,Первый срок часть второго даже одобрял его дей...,3,БОЛЕЕ ДВУХ СРОКОВ
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach,26,думаю много весёлого было Например когда 94-ом...,1,МММ
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach,7,какой бывших республик СССР была проведена люс...,1,СССР


## 5. Удаляем посты, где не оказалось ни одного капса

In [41]:
# удаляем посты, где не оказалось ни одного капса
all_dfs = all_dfs[all_dfs.n_caps != 0]
print(len(all_dfs))
all_dfs.reset_index(drop=True, inplace=True)
all_dfs.sex.value_counts()

36510


2    27173
1     9328
0        9
Name: sex, dtype: int64

In [42]:
# мужчины в 3 раза чаще используют капслок, чем женщины

In [43]:
all_dfs[all_dfs.source == "yajemati"].sex.value_counts()
# даже в женском паблике мужчин с капслоком больше, чем женщин
# ПРОВЕРИТЬ ПОТОМ ОСМЫСЛЕННЫЙ КАПСЛОК 

1    3314
2    3276
0       2
Name: sex, dtype: int64

## 6. Попробуем обработать группы: добавим в новую колонку activity групп юзера

In [46]:
group_info = all_dfs.group_info.values.tolist()

In [47]:
all_dfs.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source,n_tokens,tokens,n_caps,caps
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach,61,это был еще конец только декабря 1999 года пон...,1,КГБ
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,6,Лента сама так решила СЛАВУ ИМПЕРАТОРА,2,СЛАВУ ИМПЕРАТОРА
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,22,Первый срок часть второго даже одобрял его дей...,3,БОЛЕЕ ДВУХ СРОКОВ
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach,26,думаю много весёлого было Например когда 94-ом...,1,МММ
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach,7,какой бывших республик СССР была проведена люс...,1,СССР


In [48]:
len(group_info)

36510

In [49]:
group_info[3]

nan

In [50]:
all_activities = []
for i,groups in enumerate(group_info):
    if not isinstance(groups, float):
        groups = groups.split(" -|-|- ")
        #activities = [j.split(" -||- ")[-1] if not isinstance(j, float) else [] for j in groups]
        activities = [j.split(" -||- ")[-1] for j in groups]
        all_activities.append(activities)
    else:
        all_activities.append([])

In [51]:
len(all_activities)

36510

In [52]:
all_activities = [" | ".join(i) for i in all_activities]

In [53]:
all_dfs["activity"] = all_activities 

In [54]:
all_dfs.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source,n_tokens,tokens,n_caps,caps,activity
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach,61,это был еще конец только декабря 1999 года пон...,1,КГБ,
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,6,Лента сама так решила СЛАВУ ИМПЕРАТОРА,2,СЛАВУ ИМПЕРАТОРА,Kindergarten | Fictional Character | Internet ...
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,22,Первый срок часть второго даже одобрял его дей...,3,БОЛЕЕ ДВУХ СРОКОВ,Internet Media | Internet Media | Society | Ed...
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach,26,думаю много весёлого было Например когда 94-ом...,1,МММ,
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach,7,какой бывших республик СССР была проведена люс...,1,СССР,Internet Media | Internet Media | Internet Med...


In [55]:
all_dfs.to_csv("temp.csv", index=False)

KeyboardInterrupt: 

In [7]:
df = pd.read_csv("temp.csv")
df.head()

,bdate,city,comment_id,comment_text,first_name,from_id,group_info,last_name,parent_id,parent_text,sex,source,n_tokens,tokens,n_caps,caps,activity
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,NaN,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,lentach,61,это был еще конец только декабря 1999 года пон...,1,КГБ,NaN
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,PidorasBlog -||- паблик чечни -||- Магнит гомо...,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,6,Лента сама так решила СЛАВУ ИМПЕРАТОРА,2,СЛАВУ ИМПЕРАТОРА,Kindergarten | Fictional Character | Internet ...
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Лентач -||- Пропаганда здравого смысла -||- -...,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,lentach,22,Первый срок часть второго даже одобрял его дей...,3,БОЛЕЕ ДВУХ СРОКОВ,Internet Media | Internet Media | Society | Ed...
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,NaN,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,lentach,26,думаю много весёлого было Например когда 94-ом...,1,МММ,NaN
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Лентач -||- Пропаганда здравого смысла -||- -...,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,lentach,7,какой бывших республик СССР была проведена люс...,1,СССР,Internet Media | Internet Media | Internet Med...


In [56]:
df = all_dfs
df = df.drop(columns="group_info")

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36510 entries, 0 to 36509
Data columns (total 16 columns):
bdate           28719 non-null object
city            36500 non-null object
comment_id      36510 non-null object
comment_text    36510 non-null object
first_name      36510 non-null object
from_id         36510 non-null object
last_name       36509 non-null object
parent_id       36510 non-null object
parent_text     24549 non-null object
sex             36510 non-null object
source          36510 non-null object
n_tokens        36510 non-null int64
tokens          36510 non-null object
n_caps          36510 non-null int64
caps            36510 non-null object
activity        36510 non-null object
dtypes: int64(2), object(14)
memory usage: 4.5+ MB


## 7. Посчитаем, сколько раз встречаются слова, написанные капслоком, в Википедии, в капслоке и без капслока. 
_слово написано капсом / (слово написано капсом + слово написано не капсом)_

In [8]:
caps = df.caps.values.tolist()
caps = [i.split() for i in caps]
caps_list = list(itertools.chain.from_iterable(caps))

KeyboardInterrupt: 

In [2]:
with open("caps.txt", "r") as f:
    caps_list = [i.strip() for i in f.readlines()]

In [5]:
files = os.listdir("./WikiExtractor/")
files = [i for i in files if not i.startswith(".") and not i.endswith((".tsv", ".py")) and i not in ["wiki-files", "wiki.txt"]]
print(files)

['kekbfaa', 'kekasab', 'kekbfab', 'kekasaa']


In [6]:
for file in files:
    
    with codecs.open("./WikiExtractor/" + file, "r", "utf-8", "replace") as f:
        f = f.readlines()
    print(len(f))
            
    all_tokens = []
    for line in f:    
        try:
            tokens = [_.text for _ in list(tokenize(line))] 
            all_tokens += tokens
        except StopIteration and RuntimeError:
            continue
    
    tokens_count_dict = dict(zip(*np.unique(all_tokens, return_counts=True)))
    print(file, "counts collected")
    
    caps_dict = {i:[] for i in caps_list}
    
    for word in caps_dict:
        try:
            upper_count = tokens_count_dict.get(word,0)
            lower_count = tokens_count_dict.get(word.lower(),0) + tokens_count_dict.get(word.capitalize(),0)
            total_count = upper_count + lower_count
            metrics = upper_count / total_count
            caps_dict[word] = (metrics, total_count)
        except:
            caps_dict[word] = (0, 0)
        
    with open("./WikiExtractor/" + file + ".tsv", "w") as k:
        for key,v in caps_dict.items():
            k.write(key + "\t" + str(v[0]) + "\t" + str(v[1]) + "\n")

256374
kekbfaa counts collected
247961
kekasab counts collected
217899
kekbfab counts collected
293609
kekasaa counts collected


Далее см. choose-meaningful-capslock.ipynb

## 8. Посчитаем количество осмысленного капслока в каждом комментарии

In [58]:
with open("unusual_caps.txt", "r") as f:
    f = f.readlines()
    f = [i.strip() for i in f]
print(len(f))

17956


In [59]:
f[:10]

['10-ЛЕТНИЙ',
 'АААА',
 'ААААА',
 'АААААА',
 'ААААААА',
 'АББА',
 'АБДУЛА',
 'АБДУЛОВ',
 'АБДУЛЬ',
 'АБОИМ']

In [60]:
caps = [i.split() for i in df.caps.values.tolist() if i != ""]
only_mean_caps = []
n_mean_caps = []

In [61]:
for i, text in enumerate(caps):
    mean_caps = [j for j in text if j in f]
    only_mean_caps.append(" ".join(mean_caps))
    n_mean_caps.append(len(mean_caps))

In [62]:
df["mean_caps"] = only_mean_caps

In [63]:
df["n_mean_caps"] = n_mean_caps

In [64]:
len(df)

36510

In [65]:
len(df[df.n_mean_caps != 0])

18832

# Обрабатываем даты - переводим в возраст

In [66]:
from datetime import date

class Born:
    def __init__(self, date):
        day, month, year = date.split(".")
        self.day = int(day.lstrip("0"))
        self.month = int(month.lstrip("0"))
        self.year = int(year)
        
def calculate_age(birth_date):
    b = Born(birth_date)
    
    try:
        born = date(b.year, b.month, b.day)
    except ValueError:
        born = date(b.year, b.month+1, 1)
    today = date.today()
    try: 
        birthday = born.replace(year=today.year)
    except ValueError: # raised when birth date is February 29 and the current year is not a leap year
        birthday = born.replace(year=today.year, month=born.month+1, day=1)
    if birthday > today:
        return today.year - born.year - 1
    else:
        return today.year - born.year

In [67]:
birthdates = df.bdate.values.tolist()
age = []
birth_years = []
for b in birthdates:
    if str(b).count(".")==2:
        age.append(calculate_age(b))
        birth_years.append(Born(b).year)
    else:
        age.append(None)
        birth_years.append(None)

len(birth_years)

36510

In [68]:
birth_years[3]

1998

In [69]:
age[3]

21

In [70]:
df["age"] = age
df["birth_year"] = birth_years

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36510 entries, 0 to 36509
Data columns (total 20 columns):
bdate           28719 non-null object
city            36500 non-null object
comment_id      36510 non-null object
comment_text    36510 non-null object
first_name      36510 non-null object
from_id         36510 non-null object
last_name       36509 non-null object
parent_id       36510 non-null object
parent_text     24549 non-null object
sex             36510 non-null object
source          36510 non-null object
n_tokens        36510 non-null int64
tokens          36510 non-null object
n_caps          36510 non-null int64
caps            36510 non-null object
activity        36510 non-null object
mean_caps       36510 non-null object
n_mean_caps     36510 non-null int64
age             16999 non-null float64
birth_year      16999 non-null float64
dtypes: float64(2), int64(3), object(15)
memory usage: 5.6+ MB


In [72]:
df["caps_frac"] = df.n_caps/df.n_tokens

In [73]:
df["caps_mean_frac"] = df.n_mean_caps/df.n_tokens

In [74]:
df.head()

,bdate,city,comment_id,comment_text,first_name,from_id,last_name,parent_id,parent_text,sex,...,tokens,n_caps,caps,activity,mean_caps,n_mean_caps,age,birth_year,caps_frac,caps_mean_frac
0,24.9,Saint Petersburg,11736787,Но это был еще не конец. Я только 31 декабря 1...,Yulia,6343530,Vershinina,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",1,...,это был еще конец только декабря 1999 года пон...,1,КГБ,,,0,NaN,NaN,0.016393,0.000000
1,31.8,Novosibirsk,11736862,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,Alexey,135863776,Krutikov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,...,Лента сама так решила СЛАВУ ИМПЕРАТОРА,2,СЛАВУ ИМПЕРАТОРА,Kindergarten | Fictional Character | Internet ...,СЛАВУ ИМПЕРАТОРА,2,NaN,NaN,0.333333,0.333333
2,5.8,Novosibirsk,11736889,Первый срок (и часть второго даже) я одобрял е...,Maxim,10862032,Gerasimov,11736701.0,"20 лет назад, 9 августа 1999 года, президент Р...",2,...,Первый срок часть второго даже одобрял его дей...,3,БОЛЕЕ ДВУХ СРОКОВ,Internet Media | Internet Media | Society | Ed...,БОЛЕЕ ДВУХ СРОКОВ,3,NaN,NaN,0.136364,0.136364
3,18.6.1998,Saint Petersburg,11736884,я думаю много весёлого было. Например когда в ...,Alexey,29670766,Kovalenko,[11736731],Зря боря подарил стране плешивого фашиста .,2,...,думаю много весёлого было Например когда 94-ом...,1,МММ,,,0,21.0,1998.0,0.038462,0.000000
4,26.7,Novosibirsk,11737428,в какой из бывших республик СССР была проведен...,Pavel,320342320,Vladimirovich,[11736787],Но это был еще не конец. Я только 31 декабря 1...,2,...,какой бывших республик СССР была проведена люс...,1,СССР,Internet Media | Internet Media | Internet Med...,,0,NaN,NaN,0.142857,0.000000


## Сохраняем итоговые результаты в файлы

In [13]:
df = pd.read_csv("capitalization.csv")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36510 entries, 0 to 36509
Data columns (total 23 columns):
bdate             28719 non-null object
city              36500 non-null object
comment_id        36510 non-null int64
comment_text      36510 non-null object
first_name        36510 non-null object
from_id           36510 non-null int64
last_name         36509 non-null object
parent_id         36510 non-null object
parent_text       24549 non-null object
sex               36510 non-null int64
source            36510 non-null object
n_tokens          36510 non-null int64
tokens            36510 non-null object
n_caps            36510 non-null int64
caps              36510 non-null object
activity          28038 non-null object
mean_caps         17247 non-null object
n_mean_caps       36510 non-null int64
age               16999 non-null float64
birth_year        16999 non-null float64
caps_frac         36510 non-null float64
caps_mean_frac    36510 non-null float64
country      

In [4]:
with open("russian_cities.tsv", "r") as f:
    f = f.readlines()
russian_cities = []
for i in f:
    russian_cities.append(i.split("\t")[0])

In [9]:
df["country"] = None
for i in df.index:
    if df.city[i] in russian_cities:
        df.loc[i, "country"] = "Russia"
    elif df.city[i] == None:
        df.loc[i, "country"] = None
    else:
        df.loc[i, "country"] = "Other"

In [20]:
df[(df.age < 20)&(df.n_mean_caps > 0)]

,bdate,city,comment_id,comment_text,first_name,from_id,last_name,parent_id,parent_text,sex,...,n_caps,caps,activity,mean_caps,n_mean_caps,age,birth_year,caps_frac,caps_mean_frac,country
18,8.6.2000,Kirov,11736167,ООО ОФИГЕТЬ ООО ОФ ГЕННТЬ ООООО,Danil,540317018,Yuferev,11736005.0,Oxxxymiron выступил в поддержку митинга 10 авг...,2,...,5,ООО ОФИГЕТЬ ООО ГЕННТЬ ООООО,NaN,ОФИГЕТЬ ГЕННТЬ ООООО,3,19.0,2000.0,1.000000,0.600000,Russia
73,25.6.2005,Chişinău,11732943,Обратите внимание. Один попсовый кал для людей...,Nikita,57087473,Melman,11732555,"Собрали для вас исполнителей, которые выступят...",1,...,1,КРОВОСТОК,Religion | Internet Media | Humor | Creative W...,КРОВОСТОК,1,14.0,2005.0,0.033333,0.033333,Other
83,13.9.2000,Habana,11735000,НУ БЛИН!КУДА СКАТИЛАСЬ ОППОЗИЦИЯ?!СЛУШАТЬ ФЕЙС...,Andrey,391601839,Vlasov,11732555,"Собрали для вас исполнителей, которые выступят...",2,...,6,БЛИН КУДА СКАТИЛАСЬ ОППОЗИЦИЯ СЛУШАТЬ ФЕЙСА,"Humor | Education | Technology, Electronics | ...",БЛИН КУДА СКАТИЛАСЬ ОППОЗИЦИЯ СЛУШАТЬ ФЕЙСА,6,18.0,2000.0,0.857143,0.857143,Other
114,4.2.2005,Oryol,11731360,— Алло! Милиция?! ! ! !\n— Да.\n— Приезжайте с...,Alexander,335981395,Korolyov,11731237.0,"Совет Федерации заверил, что протестные акции ...",2,...,2,ЕХАТЬ КУДА,Movies | Internet Media | Humor | Job Search |...,ЕХАТЬ КУДА,2,14.0,2005.0,0.181818,0.181818,Russia
162,12.9.1999,Vologda,11733012,"АХАХАХА, чорт, как орно и в то же время годно))",Alexander,125068814,Kirillov,11729129,"Ты должен был бороться со злом, а не примкнуть...",2,...,1,АХАХАХА,NaN,АХАХАХА,1,19.0,1999.0,0.166667,0.166667,Russia
206,1.12.2005,Moscow,11728521,ФСЕ НАЦЫЫ РАВНЫ!ВЫ ШО ТВАРИТЕ?,Fedor,327124419,Abolemov,11727641.0,• Росстат сообщил о резком росте цен на чёрный...,2,...,4,ФСЕ НАЦЫЫ РАВНЫ ТВАРИТЕ,Internet Media | Internet Media | History | In...,ФСЕ НАЦЫЫ РАВНЫ ТВАРИТЕ,4,13.0,2005.0,1.000000,1.000000,Russia
213,29.7.2005,Moscow,11733185,он про ТРАМПА,Ilyushka,326587985,Rakin,[11727404],"Интересно, а в соседней стране, название говор...",2,...,1,ТРАМПА,Youth Organization | Discussion Сlub | Softwar...,ТРАМПА,1,14.0,2005.0,0.500000,0.500000,Russia
222,11.8.2001,Odintsovo,11726845,АХХПХАХА,Stepan,423108756,Volkov,[11726638],Уроды в масках пытаются нас запугать.\nВыходит...,2,...,1,АХХПХАХА,"Internet Media | Internet Media | Rap, Hip-Hop...",АХХПХАХА,1,18.0,2001.0,1.000000,1.000000,Russia
297,30.6.2005,Chekhov-7,11726282,а живут как свиньи и их столица выглядит хуже ...,Edward,520358379,Kinyev,[11722518],"Киргизы красавцы. Очень свободолюбивый, сильны...",2,...,1,АХАХАХАХА,NaN,АХАХАХАХА,1,14.0,2005.0,0.076923,0.076923,Other
313,9.11.2005,Almaty,11731207,Нет ну интересно же че там подвозят нашей либе...,Amir,549726532,Yamantau,[11721159],NaN,2,...,1,АХАХАХАХАХАХАХ,NaN,АХАХАХАХАХАХАХ,1,13.0,2005.0,0.047619,0.047619,Other


In [16]:
df.to_csv("capitalization.csv", index=False)

In [68]:
df1 = df.drop(columns=["comment_text", "first_name", "comment_id", "last_name", "parent_id", "parent_text", "tokens"])

In [72]:
df1.to_csv("/Users/nikolaevaanna/Desktop/capslock-new/capitalization-short.csv", index=False)

In [299]:
df.to_csv("capitalization-interests.csv", index=False)

In [301]:
len(df)

13350